In [2]:
import cv2
import numpy as np

im1=cv2.imread("12a.jpg")
im2=cv2.imread("12b.jpg")

im1=cv2.pyrDown(im1)
im2=cv2.pyrDown(im2)

#detecting points of interest and computing descriptors
detector = cv2.xfeatures2d.SIFT_create()
#detector = cv2.AKAZE_create() #Alternative if xfeature2d not available

p1, d1 = detector.detectAndCompute(im1, None)
p2, d2 = detector.detectAndCompute(im2, None)

#matching points of interest
matcher = cv2.BFMatcher(cv2.NORM_L2)
raw_matches = matcher.knnMatch(d1, trainDescriptors = d2, k = 2) 

#filtering the matches
mp1, mp2 = [], [] 
ratio=0.65
for m in raw_matches:
    if len(m) == 2 and m[0].distance < m[1].distance * ratio:
        mp1.append( p1[m[0].queryIdx] )
        mp2.append( p2[m[0].trainIdx] )
pt1 = np.float32([p.pt for p in mp1])
pt2 = np.float32([p.pt for p in mp2])

In [8]:
tr_rigid = cv2.estimateRigidTransform(pt1, pt2, False)
h=np.vstack([tr_rigid,[0,0,1]]) 
#h=cv2.findHomography(p1, p2, cv2.RANSAC, 5.0) #general transformation

In [10]:
corners = np.array([[[0,0]],[[0,im1.shape[0]]],[[im1.shape[1],0]],[[im1.shape[1],im1.shape[0]]]], dtype=np.float32)
cornersTrans = cv2.perspectiveTransform(corners,h)

mins = np.amin(cornersTrans, axis=0).astype(np.int)
maxs = np.amax(cornersTrans, axis=0).astype(np.int)

cornersAll=np.concatenate((cornersTrans,corners))

allmins = np.amin(cornersAll, axis=0).astype(np.int)
allmaxs = np.amax(cornersAll, axis=0).astype(np.int)

tH=np.dot([[1, 0, -1 * allmins[0][0]], [0, 1, -1 * allmins[0][1]], [0, 0, 1]],h)

warped_image = cv2.warpPerspective(im1, tH, (allmaxs[0][0] - allmins[0][0], allmaxs[0][1] - allmins[0][1])) 

point = np.array(-allmins[0][0],  -allmins[0][1]).astype(np.int)

output_image=np.zeros_like(warped_image)
output_image[point[1]:point[1] + im2.shape[0],point[0]:point[0] + im2.shape[1]] = im2

int_corners=[[max(cornersTrans[0][0,0],cornersTrans[1][0,0]),max(0,cornersTrans[0][0,1]-allmins[0][1],cornersTrans[2][0,1]-allmins[0][1])], [im1.shape[1],max(cornersTrans[1][0,1],cornersTrans[3][0,1])]]            

int1=output_image[int_corners[0][1]:int_corners[1][1],int_corners[0][0]:int_corners[1][0]]
int2=warped_image[int_corners[0][1]:int_corners[1][1], int_corners[0][0]:int_corners[1][0]]

fx=np.arange(int1.shape[1])/(1.*int1.shape[1])
f1=int1*(1-fx[:,np.newaxis])
f2=int2*fx[:,np.newaxis]
res=(f1+f2)

output_image+=warped_image
output_image[int_corners[0][1]:int_corners[1][1],int_corners[0][0]:int_corners[1][0]]=res.astype(np.uint8)

IndexError: too many indices for array